### The activity table shows the app installed and app purchase activities for spotify app along with country details

In [1]:
# Import and create SparkSession
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
#from pyspark.sql.types import StructField,IntegerType, StringType , DateType ,StructType, TimestampType
from pyspark.sql.functions import *
#from pyspark.sql.functions import count,sum,col, to_date
from pyspark.sql import Window

In [2]:
# Set configuration
my_conf = SparkConf()
my_conf.set("spark.app.name", "My application 1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.logConf", "false")  # Disable Spark's internal logging configuration
my_conf.set("spark.driver.log.level", "ERROR")  # Set the log level for the driver

# Create spark session
spark = SparkSession.builder.config(conf=my_conf).getOrCreate()


In [3]:
# Read csv to dataframe
df = spark.read \
    .format("csv") \
    .option("header", True) \
    .option("path", r"C:\Users\ajith\Practice Python\Pyspark_coding\Dataset\4_app_activity.txt") \
    .load()

In [4]:
df.show()
df.printSchema()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|01-01-2022|   India|
|      1| app-purchase|02-01-2022|   India|
|      2|app-installed|01-01-2022|     USA|
|      3|app-installed|01-01-2022|     USA|
|      3| app-purchase|03-01-2022|     USA|
|      4|app-installed|03-01-2022|   India|
|      4| app-purchase|03-01-2022|   India|
|      5|app-installed|03-01-2022|      SL|
|      5| app-purchase|03-01-2022|      SL|
|      6|app-installed|04-01-2022|Pakistan|
|      6| app-purchase|04-01-2022|Pakistan|
+-------+-------------+----------+--------+

root
 |-- user_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_date: string (nullable = true)
 |-- country: string (nullable = true)



In [5]:
df = df.withColumn("event_date",to_date(col("event_date"),'dd-MM-yyyy')) 

In [6]:
df.show()
df.printSchema()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+

root
 |-- user_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_date: date (nullable = true)
 |-- country: string (nullable = true)



#### 1) Find total active users each day

In [7]:
df.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



In [8]:
df.groupBy("event_date").agg(countDistinct("user_id").alias("No_of_distinct_user")).show()

+----------+-------------------+
|event_date|No_of_distinct_user|
+----------+-------------------+
|2022-01-03|                  3|
|2022-01-04|                  1|
|2022-01-01|                  3|
|2022-01-02|                  1|
+----------+-------------------+



#### 2) Find total active users each week

In [9]:
df.withColumn("Week_no",weekofyear(df.event_date)) \
  .groupBy("Week_no").agg(countDistinct("user_id").alias("No_of_distinct_user")).show()

+-------+-------------------+
|Week_no|No_of_distinct_user|
+-------+-------------------+
|      1|                  4|
|     52|                  3|
+-------+-------------------+



#### 3) Date wise total no of users who made the purchase same day they installed the app

In [10]:
df.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



In [11]:
df1 = df.groupBy("user_id","event_date").agg(countDistinct("event_name").alias("No_of_distinct_user"))  

In [12]:
df2 = df1.withColumn("new_user",when(df1["No_of_distinct_user"] == 2, df.user_id).otherwise(None))

In [13]:
df2.groupBy("event_date").agg(count(df2.new_user).alias("Date_wise_total_no_of_users")).orderBy("event_date").show()

+----------+---------------------------+
|event_date|Date_wise_total_no_of_users|
+----------+---------------------------+
|2022-01-01|                          0|
|2022-01-02|                          0|
|2022-01-03|                          2|
|2022-01-04|                          1|
+----------+---------------------------+



#### 4) Find percentages of paid users in India , USA and any other country should be tagged as others


In [14]:
df.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



In [15]:
df1 = df.filter(df.event_name == 'app-purchase') \
  .withColumn("cty",when((df.country=='India') | (df.country =='USA'),df.country).otherwise("others"))

In [16]:
df1.groupBy(df1.cty).agg((count(df1.cty)*100/df1.count()).alias("Perc")).show()

+------+----+
|   cty|Perc|
+------+----+
| India|40.0|
|others|40.0|
|   USA|20.0|
+------+----+



#### 5)Among all the users who installed the app on a given day, how many did in app purhased on the very next day -- daywise results

In [17]:
df.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



In [18]:
df.alias("act1").join(df.alias("act2"),col("act1.user_id") == col("act2.user_id"),"inner") \
                .withColumn("cnt_flag",when(datediff(col("act2.event_date"),col("act1.event_date"))==1, col("act1.user_id")).otherwise(None)) \
                .groupBy("act2.event_date").agg(count("cnt_flag")).orderBy("act2.event_date").show()

+----------+---------------+
|event_date|count(cnt_flag)|
+----------+---------------+
|2022-01-01|              0|
|2022-01-02|              1|
|2022-01-03|              0|
|2022-01-04|              0|
+----------+---------------+

